### Map of Almond Acreage by State in 2022

This notebook maps almond acres by state for the year 2022 using data from the USDA NASS Quick Stats database.

Click the badge below to open in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chuckgrigsby0/agec-370/blob/main/notebooks/09_plot_almond_acres_bearing_2022.ipynb)

We first import the necessary libraries and load the data

In [ ]:
# %pip install pygris

In [ ]:
import geopandas as gpd
import folium as folium
import pygris

import pandas as pd
import numpy as np

# Turn off scientific notation for better readability
# Similar to options(scipen=999) in R
pd.options.display.float_format = '{:.2f}'.format  # Format floats with 2 decimal places
np.set_printoptions(suppress=True)  # Suppress scientific notation in numpy arrays

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/chuckgrigsby0/agec-370/main/data/"

# Load from GitHub URL
almonds = pd.read_csv(base_url + 'almonds_acres_bearing_nass_2022.csv')
states = pygris.states(cb=True, year=2022)

In [ ]:
# Convert to string and pad with leading zeros to length 2
almonds['state_fips_code'] = almonds['state_fips_code'].astype(str).str.zfill(2)

In [ ]:
# Merge almond data with state geometries
# Use states as the left dataframe to preserve all state geometries and GeoDataFrame properties
# 'left' merge ensures all states are included even if they don't have almond data
almonds_gpd = states.merge(
    almonds, 
    left_on='STATEFP', 
    right_on='state_fips_code',
    how='left'  # Keep all states, fill missing data with NaN
)

### Create Maps Using GeoPandas

We'll create a simple choropleth map showing almond acreage bearing by state in 2022.

In [ ]:
almonds.Value.describe()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors

# Set up the figure
# figsize controls the overall size: (width, height) in inches
fig, axes = plt.subplots(1, 1, figsize=(18, 8))

# Get all states for background (to show states without data)
# Filter non-contiguous states/territories
remove_states_background = ['Alaska',
                            'American Samoa',
                            'Puerto Rico',
                            'United States Virgin Islands',
                            'Hawaii',
                            'Guam',
                            'Commonwealth of the Northern Mariana Islands']

# Create background of all contiguous US states
all_states_background = almonds_gpd[~almonds_gpd['NAME'].isin(remove_states_background)]

# First, plot ALL states as a background layer in light gray
# This ensures all state boundaries are visible
all_states_background.plot(
    ax=axes,
    color='#f0f0f0',        # Very light gray for states without data (non-producing)
    edgecolor='white',      # White borders for clean separation
    linewidth=1.0           # Medium line width for visibility
)

# Filter almond data to contiguous US only
remove_states_data = ['ALASKA', 'HAWAII']
almonds_filtered = almonds[~almonds['state_name'].isin(remove_states_data)].copy()

# Create a cleaner title by extracting the commodity description
title = 'Almonds - Acres Bearing'

# Convert to thousands of acres for better readability on the legend
almonds_filtered['Value_thousands'] = almonds_filtered['Value'] / 1000

# Create bins for the classification
# Using custom bins to highlight the differences between producing states
# Bin 0: Non-producing states (handled by background - not included in data)
# Bin 1: Minimal production (< 1,000 acres)
# Bin 2: Small production (1-10 thousand acres)
# Bin 3: Medium production (10-100 thousand acres)
# Bin 4: Large production (100-500 thousand acres)
# Bin 5: Major production (> 500 thousand acres)

bins = [0, 1, 10, 100, 500, almonds_filtered['Value_thousands'].max() + 1]
labels = ['< 1k acres', '1k - 10k acres', '10k - 100k acres', 
          '100k - 500k acres', '> 500k acres']

# Create binned categories
almonds_filtered['Value_binned'] = pd.cut(
    almonds_filtered['Value_thousands'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Merge the filtered almond data with geometries for plotting
almonds_to_plot = almonds_gpd.merge(
    almonds_filtered[['state_fips_code', 'Value_binned']],
    left_on='STATEFP',
    right_on='state_fips_code',
    how='inner'  # Only keep states with almond data
)

# Define a discrete colormap for binned data
# Using a blue-green-yellow palette that works well for categorical/binned data
# Colors progress from light (low production) to dark/vibrant (high production)
colors = ['#c7e9b4', '#7fcdbb', '#41b6c4', '#2c7fb8', '#253494']

# Create a custom categorical colormap
cmap = mcolors.ListedColormap(colors)

# Convert categorical data to numeric for plotting
# Create a mapping from category to number
category_mapping = {label: i for i, label in enumerate(labels)}
almonds_to_plot['Value_numeric'] = almonds_to_plot['Value_binned'].map(category_mapping)

# Plot the choropleth map on top of the background
# This will only color states that have data
almond_plot = almonds_to_plot.plot(
    column='Value_numeric',
    cmap=cmap,
    legend=True,
    ax=axes,
    edgecolor='white',      # White borders match background for cohesive look
    linewidth=1.0,          # Match background line width
    categorical=True,       # Treat as categorical data
    legend_kwds={
        'loc': 'lower center',      # Position legend at bottom center
        'ncol': 5,                   # Display legend items in 5 columns
        'frameon': True,             # Add frame around legend
        'framealpha': 0.9,           # Semi-transparent frame
        'fontsize': 10,              # Legend text size
        'title': 'Almond Production',  # Legend title
        'bbox_to_anchor': (0.5, -0.15)  # Position below the map
    }
)

# Update legend labels to show the actual bin ranges
legend = axes.get_legend()
if legend:
    for i, text in enumerate(legend.get_texts()):
        if i < len(labels):
            text.set_text(labels[i])

# Set the title for the map
axes.set_title(
    f'{title} (2022)',
    fontsize=16,
    fontweight='bold',
    pad=20  # Space between title and map
)

# Remove axis labels (latitude/longitude) for cleaner look
axes.set_xlabel('')
axes.set_ylabel('')

# Remove tick marks and labels for cleaner appearance
axes.set_xticks([])
axes.set_yticks([])

# Turn off the axis frame for a cleaner look
axes.axis('off')

# Add data source note at bottom
fig.text(
    0.99, 0.02,
    'Data Source: USDA National Agricultural Statistics Service (NASS)',
    ha='right',
    fontsize=9,
    style='italic',
    alpha=0.7
)

# Adjust spacing for better appearance
# Extra bottom space for the legend
plt.tight_layout(rect=[0, 0.08, 1, 0.98])  # Leave space for title, legend, and source note

# Display the figure
plt.show()